```
process #pragma templet ...  in 'parsweep.hpp' file (reload the file after processing, if needed)
```

In [ ]:
!../../bin/acta parsweep.hpp

```
get Everest access token using password authentication
```

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
{
    templet::everest_engine teng("everest-login","everest-password");
    teng.save_access_token();
    if (teng) { string token; teng.get_access_token(token); std::cout << token;}
    else {std::cout << "--- authentication failed ---";}
}

```
get Everest application(s) description using access token and application name
```

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
{
    templet::everest_engine teng("everest-access-token");
    teng.print_app_description("lyapunov");
}

```
run parsweep sample
```

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#pragma cling add_include_path("../../lib/")
#include "parsweep.hpp"
#include <chrono>
#include <random>
{
   
	app_ID = "everest-application-ID";
    maple_code = "./everest-app/code.txt";
	
	NUM_TASKS = 2;
	// set NUM_WORKERS in 'parsweep.hpp' if needed

	RHO = new double[NUM_TASKS];
	SIGMA = new double[NUM_TASKS];
	BETA = new double[NUM_TASKS];
    
    std::random_device rd;
    std::mt19937 gen(rd());
    
    std::uniform_real_distribution<> dist_rho(-1.0,1.0);
    std::uniform_real_distribution<> dist_sigma(-1.0,1.0);
    std::uniform_real_distribution<> dist_beta(-1.0,1.0);
    
    for(int i=0; i<NUM_TASKS; i++){
        RHO[i]   = 28.0 + dist_rho(gen);
        SIGMA[i] = 10.0 + dist_sigma(gen);
        BETA[i]  = 2.66666 + dist_beta(gen);
    }

	templet::engine eng;
	templet::everest_engine teng("everest-access-token");
 
	master mst(eng);
	worker wks[NUM_WORKERS];

	for (int i = 0; i < NUM_WORKERS; i++) {
		wks[i].engines(eng, teng);
		mst.r(wks[i].r);
	}
    
    auto start = std::chrono::high_resolution_clock::now();
	eng.start();
	teng.run();
    auto end = std::chrono::high_resolution_clock::now();
	
   if (!eng.stopped()) {
		templet::everest_error err;
		if (teng.error(&err)) templet::everest_engine::print_error(&err);
		else cout << "Logical error(((";
	}
    else{ 
        cout << "Success!!!\n";
        std::chrono::duration<double> diff = end-start;
        cout << "Computation time is " << diff.count() << " seconds";
    }
}

```
compile 'parsweep.cpp' file
```

In [ ]:
!g++ parsweep.cpp -l curl -I ../../lib -I ../../lib/opt -o parsweep

```
run 'parsweep'
```

In [ ]:
!./parsweep